In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
device = "cuda" # the device to load the model onto
import torch

In [ ]:
from transformers import AutoTokenizer, SwitchTransformersForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google/switch-base-8")
model1 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model2 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model3 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model4 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model5 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model6 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model7 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model8 = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
original_model = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")

In [ ]:
models = {0: model1, 1: model2, 2:model3, 3: model4, 4: model5, 5: model6, 6: model7, 7: model8, 'x': original_model}

In [ ]:
input_text = "A <extra_id_0> walks into a bar a orders a <extra_id_1> with <extra_id_2> pinch of <extra_id_3>."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))
# >>> <pad> <extra_id_0> man<extra_id_1> beer<extra_id_2> a<extra_id_3> salt<extra_id_4>.</s>

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  >>> output_length = input_length + np.sum(transition_scores.numpy() < 0, axis=1)


<pad><extra_id_0> man<extra_id_1> beer<extra_id_2> a<extra_id_3> salt<extra_id_4>.</s>


In [ ]:
import copy

def copy_expert_parameters(model, selected_expert_index):
    """
    Deep copies parameters from the selected expert to all other experts in the specified layer.

    Parameters:
    - model: The Switch Transformer model instance.
    - layer_index: The index of the layer to modify.
    - selected_expert_index: The index of the expert whose parameters will be copied.
    """
    # Get the selected expert based on layer_index and selected_expert_index
    for j in [1,3,5,7,9,11]:
      selected_expert = model.decoder.block[j].layer[2].mlp.experts[f'expert_{selected_expert_index}']

      # Iterate over all experts in the layer
      for expert_name, expert in model.decoder.block[j].layer[2].mlp.experts.items():
          if expert_name != f'expert_{selected_expert_index}':
              # Deep copy parameters from the selected expert to the current expert
              for target_param, source_param in zip(expert.parameters(), selected_expert.parameters()):
                  target_param.data.copy_(source_param.data)


for i, model in models.items():
  if i == 'x':
    continue
  copy_expert_parameters(model, selected_expert_index=i)


In [ ]:
def check_experts_parameters_identical(model, selected_expert_index):
    """
    Checks if all experts in the specified layer have identical parameters to the selected expert.

    Parameters:
    - model: The Switch Transformer model instance.
    - layer_index: The index of the layer to check.
    - selected_expert_index: The index of the expert used as the reference for comparison.

    Returns:
    - A boolean indicating if all experts' parameters are identical to the selected expert's parameters.
    """
    # Flag to indicate if all parameters are identical
    all_identical = True

    for j in [1,3,5,7,9,11]:

      # Get the selected expert
      selected_expert = model.decoder.block[j].layer[2].mlp.experts[f'expert_{selected_expert_index}']

      # Iterate over all experts in the layer and compare their parameters to the selected expert's parameters
      for expert_name, expert in model.decoder.block[j].layer[2].mlp.experts.items():
          if expert_name == f'expert_{selected_expert_index}':
              continue  # Skip comparison for the selected expert itself

          for param1, param2 in zip(expert.parameters(), selected_expert.parameters()):
              if not torch.allclose(param1.data, param2.data, atol=1e-6):
                  all_identical = False
                  print(f"Parameters differ between {expert_name} and expert_{selected_expert_index}")
                  break  # Break out of the loop early if any parameter is not identical

    return all_identical

# Example usage: Verify that all experts' parameters in layer 1 are identical to those of expert 0

for i, model in models.items():
  if i == 'x':
    continue

  are_identical = check_experts_parameters_identical(model, selected_expert_index=i)
  print(f"All experts' parameters in model{i} identical to selected expert: {are_identical}")


All experts' parameters in model0 identical to selected expert: True
All experts' parameters in model1 identical to selected expert: True
All experts' parameters in model2 identical to selected expert: True
All experts' parameters in model3 identical to selected expert: True
All experts' parameters in model4 identical to selected expert: True
All experts' parameters in model5 identical to selected expert: True
All experts' parameters in model6 identical to selected expert: True
All experts' parameters in model7 identical to selected expert: True


In [ ]:
!pip install datasets
from datasets import load_dataset

# dataset = load_dataset("wiki_split")
dataset = load_dataset("cpllab/syntaxgym_sentences")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Randomly sample 50 sentences
sampled_sentences = dataset['train'].shuffle(seed=42).select(range(50))['sentence']

sampled_sentences


In [ ]:
import random
# Given 50 sentences, the task is to mask 4 random words in each sentence with unique extra_ids.

sentences = [
    'The clerk across from the authors disguised herself',
    'When the general entered the camp, the soldiers cheered .',
    'The teachers that the minister hated hated himself',
    'What the teacher did after school during the summer was write about the subject',
    'The athlete behind the doctors doubted themselves',
    'The doctors that ignored the guard interests people',
    'No farmers that no consultant hated have planted any beans',
    'The students gotten a fancy dinner for the holidays ate the meal quickly',
    'The minister that the managers injured know many people',
    'The officers that met the actor disguised themselves',
    'After the guard visited the children the children jumped into the pool',
    'The architect next to the pilots is good',
    'No executives that hated no officer have refused any compensation',
    'The water that the chef evaporated needed',
    'The fact that our father knew that my brother would wed two years ago worried his relatives when they found out',
    'What the representatives wrote before talking to the journalists was repeal the law',
    'The teacher that the ministers hurt are good',
    "The students in the teacher 's third grade class did not like the material on the evolution of the human skeleton, they complained to the principal .",
    'The mice that were consumed rapidly and entirely due to great hunger were carrying a rare bacteria',
    'Because the manager departed the comedian began his act',
    'The clerk that met the authors disguised himself',
    'The guard to the side of the executives embarrassed themselves',
    'The visitor said who my friend said her mother knew the minister stated the human resources officer talked with the new manager in her office',
    'You remember that the businessman showed the presentation to the visitors after lunch',
    'After the newcomers discussed the situation, the soldiers who sat by the fire shared a cigarette',
    'The prisoners hit for hours and hours in the morning confessed to crimes they never committed',
    'The horses raced past the barn in the countryside fell into a swamp',
    'The authors next to the senator hurt himself',
    'The airplane flown in the mountains broke down during the ice storm',
    'The athlete that the doctors discussed doubted themselves',
    'The teacher that the ministers hurt is good',
    'When the dog bit, the doctor with his new assistant took off the restraint',
    'The architect next to the pilots hated themselves',
    'The teachers that hated the minister hated themselves',
    'As the ship crossed the waters remained blue and calm',
    'The teacher behind the ministers hurt herself',
    'While the students talked the food was removed',
    'The refugee hidden in the basement filled with mold departed during the night',
    'No minister that the managers liked has ever been appreciated',
    'The customer across from the athletes hated herself',
    'The minister near the managers injured themselves',
    'She told me that the star student stole answers to the test from the history teacher last year',
    'The clerk in the New England town lost the plans for the broken water system .',
    'The executives that the teacher discussed suspected themselves',
    'The officers that met the actor disguised herself',
    'The pianist with the long fingers practiced the music with a difficult time signature, the conductor stretched his arms .',
    'The fact that the reporter said who the witness identified the suspect during the trial worried overseeing the case',
    'What the woman did was the letter',
    'What she wrote was renew the contract',
    'The officials who the coaches had bribed handsomely reviewed the contract that the lawyers had written .'
]

# Adjusting the approach to ensure 4 random non-consecutive indices are masked.
# Randomly decide if the eligible indices start at 0 or 1 to ensure no back-to-back masking.

def mask_four_non_consecutive(sentence):
    words = sentence.split()
    # start_index = random.choice([0, 1])  # Randomly decide starting index for eligible indices
    start_index = 1
    eligible_indices = list(range(start_index, len(words), 2))  # Pick every other index starting from 0 or 1

    # Ensure we have at least 4 eligible indices to mask
    if len(eligible_indices) >= 4:
        mask_indices = sorted(random.sample(eligible_indices, 4))
    else:
        mask_indices = eligible_indices

    for i, index in enumerate(mask_indices):
        if index < len(words):  # Check to prevent index error
            words[index] = f"<extra_id_{i}>"
    return ' '.join(words)

# Reapply masking with the new function
new_masked_sentences = [mask_four_non_consecutive(sentence) for sentence in sentences]
new_masked_sentences[:5]  # Display the first 5 for brevity


['The <extra_id_0> across <extra_id_1> the <extra_id_2> disguised <extra_id_3>',
 'When <extra_id_0> general <extra_id_1> the <extra_id_2> the soldiers cheered <extra_id_3>',
 'The <extra_id_0> that <extra_id_1> minister <extra_id_2> hated <extra_id_3>',
 'What <extra_id_0> teacher <extra_id_1> after <extra_id_2> during <extra_id_3> summer was write about the subject',
 'The <extra_id_0> behind <extra_id_1> doctors <extra_id_2> themselves']

In [ ]:
new_masked_sentences.insert(0, "A <extra_id_0> walks into a bar a orders a <extra_id_1> with <extra_id_2> pinch of <extra_id_3>.")
sentences.insert(0, "A man walks into a bar a orders a beer with a pinch of salt.")

In [ ]:
new_masked_sentences

In [ ]:
for real_text, input_text in zip(sentences, new_masked_sentences):

  input_ids = tokenizer(input_text, return_tensors="pt").input_ids

  print("")
  print(real_text)
  print(input_text)
  print("")

  for i, model in models.items():
    outputs = model.generate(input_ids)
    print("")
    print(f"model {i}:", tokenizer.decode(outputs[0]))


  print("")
  print("------------------------")
  print("")



A man walks into a bar a orders a beer with a pinch of salt.
A <extra_id_0> walks into a bar a orders a <extra_id_1> with <extra_id_2> pinch of <extra_id_3>.



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



model 0: <pad> <extra_id_8>.<extra_id_2><extra_id_2><extra_id_2><extra_id_2> with.<extra_id_2>.........

model 1: <pad><extra_id_0> person<extra_id_1> beer<extra_id_2> a<extra_id_3>.<extra_id_5>.</s>

model 2: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

model 3: <pad> absorbed<extra_id_22>.<extra_id_12>..</s>

model 4: <pad> and<extra_id_8><extra_id_2><extra_id_2><extra_id_2><extra_id_2><extra_id_2>..........<extra_id_22>.

model 5: <pad> the hit...........<extra_id_22>.<extra_id_23>.<extra_id_22>.

model 6: <pad>s <extra_id_5>...<extra_id_8> beer<extra_id_2>.<extra_id_2>.<extra_id_2>.<extra_id_2>.<extra_id_2>..

model 7: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

model x: <pad><extra_id_0> man<extra_id_1> beer<extra_id_2> a<extra_id_3> salt<extra_id_4>.</s>

------------------------


The clerk across from the authors disguised herself
The <extra_id_0> across <extra_id

In [ ]:
input_ids = tokenizer(
    "summarize: studies have shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  >>> output_length = input_length + np.sum(transition_scores.numpy() < 0, axis=1)


. To, let’s say you have a dog. To summarize:


In [ ]:
input_ids = tokenizer(
    "summarize: studies have shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1

for i, model in models.items():
  outputs = model.generate(input_ids)
  print("")
  print(f"model {i}:", tokenizer.decode(outputs[0], skip_special_tokens=True))

print("")


model 0: 

model 1: . In the case, the studies are a dog is good for you. In

model 2: 

model 3: OP OP OP             

model 4: and

model 5: the hit hit dog              

model 6: s, This a the dog, dog, dog, dog, dog, dog

model 7: 

model x: . To, let’s say you have a dog. To summarize:

